### Dictionaries, APIs and JSON
#### Dictionaries, like lists and sets, are a type of **collection**  
**properties** are **key-value** pairs

- **dict1.update(dict2)** merge two dictionaries

- **list.remove(item)**. removes the specified item

- **list(dict.keys())** returns list of dict keys

- **del dict[key]** deletes dictionary key

**Application Programming Interface (API)** is a web application that serves data upon request to other web apps.
- API's are typically very content specific: weather, jokes, cooking recipes, stock prices, etc.
- API's typically have a website where you can read up on what kind of data is available, and what kind of requests can be made. Often you can get data by category--or even at random.
- **HTTP Requests** to a server receive a **response** as web page of API data
- **GET** is the **method** by which some HTTP Requests are made
- **JavaScript Object Notation (JSON)** is a standard format for transmitting data on the Web
- **JSON** is practically identical to Python dictionaries

**loading data from an API**.

**requests** module is the required package for loading data from API's

In [1]:
# install ipywidgets
%pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [2]:
# import requests module
import requests
import pprint as pp
import html, re, unicodedata, random
# import widgets for making buttons for trivia choices
import ipywidgets as widgets
# must be installed: pip install ipywidgets
from IPython.display import Image, display
import sys

In [3]:
# should point into .../venv/...
print(widgets.__version__) # 8.1.8

8.1.8


In [4]:
# make a widget button:
test_btn = widgets.widgets.Button(description="Click Me")
# make an output widget:
outputter = widgets.widgets.Output()

In [5]:
# make a func for the button to call when clicked
# pass the button itself which is calling into the func as `b`
def handle_btn_click(b):
    with outputter:
        outputter.clear_output(wait=True)
        print("You clicked the button!")

In [6]:
# instruct the btn to call the func:
test_btn.on_click(handle_btn_click)

In [7]:
# display the btn; whtn the btn is clicked, the output appears below it
display(test_btn, outputter)

Button(description='Click Me', style=ButtonStyle())

Output()

**https://catfact.ninja/**  
for setting cat fact options (category, number of questions, etc.)

In [8]:
# API URL:
# API requests are made to a URL, just like if you want to order a pizza you have
# to call some number or go to some website
# We will be requesting a "Cat Fact" from:
url = "https://catfact.ninja/fact"
# Open a new browser tab and paste just the URL string part into the address bar
# We land on the catfact page where a random "Cat Fact" appears in JSON format
# copy-paste the "Cat JSON" and save it as a string called cat_fact_json
# Use single quotes around the url, as the JSON itself contains a lot of double quotes
# "pretty print" the cat facts json / dict:

# We see that the structure is very much like a Python dictionary

**json()** format is just like Python dictionary

**make a request to the Cat Fact API and handle the response**

In [9]:
random_cat_fact = requests.get(url)
# .json() parses the response into usable dict; 
print('type(random_cat_fact):',type(random_cat_fact)) 
# <class 'requests.models.Response'>
print('random_cat_fact:',random_cat_fact)
# otherwise you just get a respons object: <Response [200]>

type(random_cat_fact): <class 'requests.models.Response'>
random_cat_fact: <Response [200]>


In [10]:
# the response 200 must be parsed / unpacked
# it's just like how we so often have to "unpack" bundled data in python:
nums = range(1,11)
print(nums)
nums = list(range(1,11))
print(nums)

range(1, 11)
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [11]:
# do the request again BUT this time UNPACK
# the result (that is PARSE it) with .json()
cat_fact_dict = requests.get(url).json()
# .json() parses the response into usable dict; 
print('type(cat_fact_dict):',type(cat_fact_dict)) 
# <class 'requests.models.Response'>
pp.pprint(cat_fact_dict)

type(cat_fact_dict): <class 'dict'>
{'fact': 'The most popular pedigreed cat is the Persian cat, followed by the '
         'Main Coon cat and the Siamese cat.',
 'length': 101}


In [12]:
# get just the cat "fact" text from the dictionary
cat_fact = cat_fact_dict["fact"]
print('type(cat_fact):',type(cat_fact)) 
pp.pprint(cat_fact)

type(cat_fact): <class 'str'>
('The most popular pedigreed cat is the Persian cat, followed by the Main Coon '
 'cat and the Siamese cat.')


In [13]:
# clean the text by removing html special entities:
# cat_fact = html.unescape(cat_fact)
# pp.pprint(cat_fact)

#### **declaring the input and output data types of a function**
By default:
- a function will accept and try to run any dtype of input
- a function will return whatever dtype of output you say
```python
def clean_text(s):
```
- The above expects a string input and logically will likely return a string output
- BUT this is not a REQUIREMENT -- the function will "try" to use ANY input

**HOWEVER!!:**
- you can set the input and output dtypes upon func definition:
```python
def clean_text(s: str) -> str:
```
- The above requires a string input and will only return a string output

In [14]:
def clean_text(s: str) -> str:
    # if the input is none return an empty string -- don't do the whole func procedure
    if s is None:
        return ""
    # a. Turn HTML entities into characters: &amp; -> &, &quot; -> "
    s = html.unescape(s)
    # b. Normalize Unicode (curly quotes, widths, etc.)
    s = unicodedata.normalize("NFKC", s)
    # c. Replace non-breaking spaces (U+00A0) with normal spaces
    s = s.replace("\xa0", " ")
    # (optional) remove zero-width spaces if present
    s = s.replace("\u200b", "")
    # d. Collapse any weird spacing
    s = re.sub(r"\s+", " ", s).strip()
    return s

In [15]:
print("RAW:", repr(cat_fact))     # shows \xa0 in the repr
clean = clean_text(cat_fact)
print("CLEAN:", clean)  

RAW: 'The most popular pedigreed cat is the Persian cat, followed by the Main Coon cat and the Siamese cat.'
CLEAN: The most popular pedigreed cat is the Persian cat, followed by the Main Coon cat and the Siamese cat.


**trivia questions API: OpenTDB.com**.


**querystings**
- In a web url the **?** separates the path from the variables being sent to the path:
    - stynax: name-value pairs "?var1=value1&var2=value2&var3=value3
    - the entire string from the **?** to the end is called the **querystring**
    - everything before the **?** is called the **base url**
    - the path is the pizza place phone number
    - then comes the **?** separator:
    - the variables are your order specs (large, half pepperoni, half mushrooms, 2 Cokes, etc)

```python
# history_trivia_url = "https://opentdb.com/api.php?amount=5&category=23&difficulty=medium&type=multiple"

# the SQL query put together from the above request would look like so:
SELECT question, correct_answer, incorrect_answers, category, difficulty, type
FROM trivia_questions
WHERE category_id = 23
  AND difficulty = 'medium'
  AND type = 'multiple'
ORDER BY RANDOM()
LIMIT 5;
```

In [16]:
# go to opentdb.com and use API interface to specify what kind / how many Q's you want
# copy provided URL and come back here and paste it:
history_trivia_url = "https://opentdb.com/api.php?amount=5&category=23&difficulty=medium&type=multiple"

# the SQL query put together from the above request would look like so:
# SELECT question, correct_answer, incorrect_answers, category, difficulty, type
# FROM trivia_questions
# WHERE category_id = 23
#   AND difficulty = 'medium'
#   AND type = 'multiple'
# ORDER BY RANDOM()
# LIMIT 5;

sport_trivia_url = "https://opentdb.com/api.php?amount=5&category=21&difficulty=hard&type=multiple"

mythology_trivia_url = "https://opentdb.com/api.php?amount=5&category=20&difficulty=hard&type=multiple"

In [17]:
# request the trivia questions; parse the result with .json()
# What does parse mean?
# Convert the <Response: 200> response object into an actual, usable dictionary
trivia = requests.get(mythology_trivia_url).json()

In [ ]:
pp.pprint(trivia)
# {'response_code': 0,
#  'results': [{'category': 'History',
#               'correct_answer': 'France',
#               'difficulty': 'medium',
#               'incorrect_answers': ['Germany', 'Italy', 'Austria'],
#               'question': 'The Battle of the Somme in World War I took place '
#                           'in which country?',
#               'type': 'multiple'},

In [28]:
# print the first question text -- the actual question not the whole dictionary
trivia_list = trivia["results"] # get just the list of trivia questions-and-answers
# pp.pprint(trivia_list[:2]) # each q-and-a is a dictionary
first_q_and_a_dict = trivia_list[0]
pp.pprint(first_q_and_a_dict)
# get the individual data nuggets out of the first Q and A dictionary:
category = first_q_and_a_dict["category"]
print(category)

correct_answer = first_q_and_a_dict["correct_answer"]
print(correct_answer)

difficulty = first_q_and_a_dict["difficulty"]
print(difficulty)

incorrect_answers = first_q_and_a_dict["incorrect_answers"]
print(incorrect_answers)

question = first_q_and_a_dict["question"]
print(question)

qtype = first_q_and_a_dict["type"]
print(qtype)
# CAUTION: do NOT declare a variable called `type`
# `type` is a RESERVED word of Ptyhon:
print(type(qtype)) # str


{'category': 'Mythology',
 'correct_answer': 'Ariadne',
 'difficulty': 'hard',
 'incorrect_answers': ['Athena', 'Ariel', 'Alana'],
 'question': 'In Greek Mythology, who was the daughter of King Minos?',
 'type': 'multiple'}
Mythology
Ariadne
hard
['Athena', 'Ariel', 'Alana']
In Greek Mythology, who was the daughter of King Minos?
multiple
<class 'str'>


In [ ]:
# semi-shortcut version of the above--extract all keys to vars of same name:
# set a bunch of newly declared vars equal to a tuple or list of corresponding values
category, correct_answer, difficulty, incorrect_answers, question, qtype = [
    first_q_and_a_dict["category"],
    first_q_and_a_dict["correct_answer"],
    first_q_and_a_dict["difficulty"],
    first_q_and_a_dict["incorrect_answers"],
    first_q_and_a_dict["question"],
    first_q_and_a_dict["type"], # don't shadow built-in 'type'
]

In [32]:
# check if above automatic var maker worked
print(category)
print(correct_answer)
print(difficulty)
print(incorrect_answers)
print(question)
print(qtype)

Mythology
Ariadne
hard
['Athena', 'Ariel', 'Alana']
In Greek Mythology, who was the daughter of King Minos?
multiple


In [34]:
# another example of mass-declaring vars as values of tuple or list

# unpacking tuple () to individual vars:
fru1, fru2, fru3 = ("apple","banana","cherry")
print(fru1)
print(fru2)
print(fru3)
print()
# unpacking list [] to individual vars:
fru1, fru2, fru3 = ["apple","banana","cherry"]
print(fru1)
print(fru2)
print(fru3)

apple
banana
cherry

apple
banana
cherry


In [35]:
all_answers = []

In [36]:
# LAB CHALLENGE:
# get all 4 answer choices for the first question into one list
# We can then loop the list and output the 4 answer choices:
# example:
# {'category': 'Mythology',
#  'correct_answer': 'Ariadne',
#  'difficulty': 'hard',
#  'incorrect_answers': ['Athena', 'Ariel', 'Alana'],
#  'question': 'In Greek Mythology, who was daughter of King Minos?',
#  'type': 'multiple'}

# procedure:
# a.) declare a new empty all_answers list (DONE ABOVE)
# b.) set all_answers list equal to the incorrect_answers (use .copy())
# c.) append the correct_answer to the all_answers list
# d.) shuffle the list OTHERWISE correct answer is ALWAYS last choice!
# e.) print the all_answers list to verify that it worked
# f.) loop the all_answers list, printing each answer as you go
#.    result is all 4 answer choices printed out, one line at a time
# g.) BONUS. precede each answer choice w letter "A", "B", "C", "D"
#.    expected outout
#       A. Ariel
#.      B. Athena
#       C. Aridne
#.      D. Alana



In [51]:
print("3 incorrect_answers:",incorrect_answers)
print("1 correct_answers:",correct_answer)
all_answers=[]
all_answers = incorrect_answers.copy()
all_answers.append(correct_answer)
random.shuffle(all_answers)
#all_answers = pd.concat([all_answers, incorrect_answers], ignore_index=True)
print("all 4 answers in same list:",all_answers)

3 incorrect_answers: ['Athena', 'Ariel', 'Alana']
1 correct_answers: Ariadne
all 4 answers in same list: ['Ariel', 'Ariadne', 'Athena', 'Alana']


In [49]:
letters = ["A", "B", "C", "D"]
lettered_answer_choice_dict = {}

**shuffle()** method for shuffling / randomizing list items

**avoid storing values in ONE list item**
- use dictionary instead where the 2 pieces of data are key:value
    - this is good:
```python
{'A': 'Ariel', 'B': 'Ariadne', 'C': 'Athena', 'D': 'Alana'}
```
- BUT the following is to be avoided, as it makes the individual data bits hard to access -- what if I just want letter(s) or answer (no letter)

```python
['A. Ariel', 'B. Ariadne', 'C. Athena', 'D. Alana']
```

In [ ]:
# print the question followed by the 4 choices on a loop:
print(question)
print()

# make and print the 4 lettered answer choices:
for letter, answer in zip(letters, all_answers):
    lettered_answer_choice = f"{letter}. {answer}"
    print(lettered_answer_choice)
    # make a property for that letter key
    lettered_answer_choice_dict[letter] = answer

print() 
# pp.pprint(lettered_answer_choice_dict)

In Greek Mythology, who was the daughter of King Minos?

A. Ariel
B. Ariadne
C. Athena
D. Alana



In [24]:
# string.split() is called on a string and returns a list
# by default it splits the list on any spaces:
# example: a question split into a list of its words:
question = "What is your name?"
# ['What', 'is', 'your', 'name?']
# split can be on other character, cuz what if there is no space
# to split on non-space, pass in split char as the delimiter
# example: a hyphenated file name split into a list of its words:
file_name = "cat-refuses-to-play-with-floppy-fish.jpg"

# ['cat', 'refuses', 'to', 'play', 'with', 'floppy', 'fish.jpg']

In [25]:
widget_output = ipywidgets.widgets.Output()
display(widget_output)

NameError: name 'ipywidgets' is not defined

In [ ]:
# define check_answer function to run when any answer button is clicked

print("Very good! The correct answer is:")
print('Not Quite! The correct answer is: "correct_answer"')

In [ ]:
# display the 4 choices in buttons.. one button per answer choice
# user just clicks a button to answer the question
# question"



The minigun was designed in 1960 by which manufacturer.



Button(description='A. Sig Sauer', style=ButtonStyle())

Button(description='B. Heckler &amp; Koch', style=ButtonStyle())

Button(description='C. Colt Firearms', style=ButtonStyle())

Button(description='D. General Electric', style=ButtonStyle())